In [4]:
from random import choices, choice
import base64
from Crypto.Cipher import AES

def Chunkerize(x, chunksize, strict = True):
    x = list(x)
    assert len(x) % chunksize == 0 if strict else True
    for n in range( len(x) // chunksize ):
        yield x[ n*chunksize : (n+1)*chunksize ]
        
def PadPlaintext(plaintext: bytes, blocksize = 16):
    npad = blocksize - len(plaintext) % blocksize
    return plaintext + bytes([npad]) * npad

def UnPadPlaintext(plaintext: bytes, blocksize = 16):
    assert type(plaintext) is bytes and len(plaintext) % blocksize == 0
    final = plaintext[-1]
    assert all( char == final for char in plaintext[-final:])
    return plaintext[:-final]

def EncryptECB(plaintext, key):
    BLOCKSIZE = 16
    assert len(key) == BLOCKSIZE
    plaintext = PadPlaintext(plaintext, BLOCKSIZE)
    ECBcipher = AES.new(key, AES.MODE_ECB)
    blocks = [ bytes(block) for block in Chunkerize(plaintext, BLOCKSIZE) ]
    cypher_blocks = [ ECBcipher.encrypt(block) for block in blocks ]
    return b''.join( ECBcipher.encrypt(block) for block in blocks )

def DecryptECB(ciphertext, key):
    ECBcipher = AES.new(key, AES.MODE_ECB)
    blocks = [ bytes(block) for block in Chunkerize(ciphertext, BLOCKSIZE) ]
    plaintext = b''.join( ECBcipher.decrypt(block) for block in blocks )
    return UnPadPlaintext(plaintext)

KEYSIZE = BLOCKSIZE = 16 # Once again, I think the attacker could just assume this

In [23]:
# Same oracle as before but prepends arbitrary bytes
# it was not specified, but I'm assuming the prepended bytes are constant because I can't image how else this would work


suffix = b'''
Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
'''

def EncryptionOracle(plaintext, verbose = False):
    key = b'B\xb7!\x9an\x92\xd6\xa2\xad\x0b\xa4\x97\x8d\xfeW\x0b'
    prefix = b"\xb48\x84t\xf7c\x13\x8f=\x9d\\\x98\x93\x13\x9c\xd3\xd7\xdc\x92\xce'\xe3\xeb\x16\xd4l\xd1\xf56\xa5\xaa\xba~\xca\xc8"
    raw_suffix = base64.decodebytes(suffix)
    new_plaintext = prefix + plaintext + raw_suffix
    print(f'Total input length: {len(plaintext)}.') if verbose else False


    ciphertext = EncryptECB(new_plaintext, key)
    return ciphertext

In [36]:
# OK so first I want to determine how long the prefix string is
# Feed  progressively longer strings in until a block repeats
plaintext = b'A'
ciphertext = EncryptionOracle(plaintext)
blocks = [ bytes(block) for block in Chunkerize(ciphertext, BLOCKSIZE) ]
while len(blocks) == len({*blocks}):
    plaintext += b'A'
    ciphertext = EncryptionOracle(plaintext)
    blocks = [ bytes(block) for block in Chunkerize(ciphertext, BLOCKSIZE) ]

repeated_block, = { block for block in blocks if blocks.count(block) > 1 }   
repeated_block_index = blocks.index(repeated_block)

# If the repeated block in in position N, the prefix must be N-1 blocks plus a little extra
prefix_len = (repeated_block_index-1) * BLOCKSIZE + ( BLOCKSIZE - len(plaintext) % BLOCKSIZE )
prefix_len

35

In [48]:
# Now calculate the length of the suffix

s = b'A' * (BLOCKSIZE - prefix_len%BLOCKSIZE)
default_len = len(EncryptionOracle(s))

while len(EncryptionOracle(s)) == default_len:
    s += b'A'

suffix_len = default_len - prefix_len - len(s)
suffix_len

138

In [51]:
# So now we have the prefix len and suffix len

prefix_pad = b'\x00' * ( BLOCKSIZE - prefix_len % BLOCKSIZE)
len(prefix_pad)

13

In [102]:
unknown = []
NBLOCKS = 9
print(f'{NBLOCKS=}')

start = prefix_len + len(prefix_pad) + (NBLOCKS-1) * BLOCKSIZE
assert start % BLOCKSIZE == 0
end = start+BLOCKSIZE

for n in range(BLOCKSIZE*NBLOCKS):
    plaintext0 = prefix_pad + b'A' * ( BLOCKSIZE*NBLOCKS - n - 1 )
    print(plaintext0)
    ciphertext0 = EncryptionOracle( plaintext0  )
    target_block = ciphertext0[start:end]
    for char in range(0xFF):
        plaintext = bytes([ *plaintext0, *unknown, char ])
        ciphertext = EncryptionOracle(plaintext)
        block = ciphertext[start:end]
        if block == target_block:
            unknown.append(char)
            print('Found %d' % char)
            break
        if char == 255:
            unknown.append(0)
    


NBLOCKS=9
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
Found 82
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
Found 111
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
Found 108
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
Found 108
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [103]:
bytes(unknown)

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x01"